In [40]:
import polars as pl
import pandas as pd
import numpy as np
import itertools

Сопоставление размеров в разных проекциях.


In [41]:
df = pd.read_csv("DF_original.csv", index_col="id")
df.head(3)

,name,comment,axial.left.anterior,axial.left.posterior,axial.left.medial,axial.left.lateral,axial.right.anterior,axial.right.posterior,axial.right.medial,axial.right.lateral,...,issue.medial.left,issue.medial.right,issue.compression.left,issue.compression.right,issue.distraction.left,issue.distraction.right,issue.distal.left,issue.distal.right,issue.mesial.left,issue.mesial.right
id,,,,,,,,,,,,,,,,,,,,,
1,Останькович В.В.,NaN,-1.0,-1.0,3.1,1.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2,Самойлова Е.Г. 29.12.2021,NaN,-1.0,-1.0,1.7,2.3,-1.0,-1.0,2.0,2.3,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
3,Авакимова М.Н.,NaN,-1.0,-1.0,8.4,2.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [43]:
projections = ["axial", "coronal", "sagittal"]
sides = ["left", "right"]
positions = ["superior", "anterior", "posterior", "lateral", "medial"]

pairs = list(map(".".join, list(itertools.product(sides, positions))))
print(pairs)

['left.superior', 'left.anterior', 'left.posterior', 'left.lateral', 'left.medial', 'right.superior', 'right.anterior', 'right.posterior', 'right.lateral', 'right.medial']


In [44]:
groups = []
for pair in pairs:
    cols = list(filter(lambda x: pair in x, df.columns))
    groups.append(cols)

print(groups)

[['sagittal.left.superior', 'coronal.left.superior'], ['axial.left.anterior', 'sagittal.left.anterior'], ['axial.left.posterior', 'sagittal.left.posterior'], ['axial.left.lateral', 'coronal.left.lateral'], ['axial.left.medial', 'coronal.left.medial'], ['sagittal.right.superior', 'coronal.right.superior'], ['axial.right.anterior', 'sagittal.right.anterior'], ['axial.right.posterior', 'sagittal.right.posterior'], ['axial.right.lateral', 'coronal.right.lateral'], ['axial.right.medial', 'coronal.right.medial']]


In [46]:
for group in groups:
    # here we fill empty data with existing one from another projection
    df[group[0]].fillna(df[group[1]], inplace=True)
    df[group[1]].fillna(df[group[0]], inplace=True)
    df.loc[df[group[0]] == -1.0, group[0]] = df[group[1]]
    df.loc[df[group[1]] == -1.0, group[1]] = df[group[0]]

    # here we calculate difference between same measures in different projections
    print(f"Difference between {group[0]} and {group[1]}")
    diff_label = f"{('.').join(group[0].split('.')[1:])}.diff"
    df[diff_label] = df.apply(
        lambda row: np.abs(row[group[0]] - row[group[1]])
        if (row[group[0]] > 0 or row[group[1]] > 0)
        else 0,
        axis=1,
    )
    print(df[[*group, diff_label]].describe(), end="\n\n")

df.to_csv("DF_diffs.csv")

Difference between sagittal.left.superior and coronal.left.superior
       sagittal.left.superior  coronal.left.superior  left.superior.diff
count              293.000000             293.000000          379.000000
mean                 2.905324               2.521945            0.406887
std                  1.242048               1.150736            0.553909
min                 -1.000000              -1.000000            0.000000
25%                  2.180000               1.800000            0.000000
50%                  2.800000               2.400000            0.200000
75%                  3.700000               3.100000            0.600000
max                  7.600000               7.300000            3.630000

Difference between axial.left.anterior and sagittal.left.anterior
       axial.left.anterior  sagittal.left.anterior  left.anterior.diff
count           290.000000              290.000000          379.000000
mean              2.198345                2.051069            0.11